In [11]:
import pandas as pd
import requests
from tqdm import tqdm
import json  # <-- Needed for converting dict/list objects to JSON strings

def get_place_recording_relationships(place_id, headers, relationship_types=None):
    """
    Fetch all place-recording relationships for a given place ID.

    Args:
        place_id (str): The MusicBrainz ID of the place.
        headers (dict): Headers to be used in the API request.
        relationship_types (list, optional): List of relationship types to filter by.

    Returns:
        list: A list of filtered relationships for the given place ID.
    """
    if not place_id or len(place_id) != 36:  # Validate UUID format
        print(f"Invalid place ID: {place_id}")
        return []

    BASE_URL = "https://musicbrainz.org/ws/2"
    endpoint = f"{BASE_URL}/place/{place_id}"
    params = {"inc": "recording-rels", "fmt": "json"}

    try:
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        if not data or "relations" not in data:
            print(f"No relations found for place ID: {place_id}")
            return []

        # Fetch all relationships
        relationships = data.get("relations", [])

        # Filter relationships if relationship_types is provided
        if relationship_types:
            relationships = [
                rel for rel in relationships if rel.get("type") in relationship_types
            ]

        # Keep only the first matching relationship, if any
        return relationships[:1] if relationships else []

    except Exception as e:
        print(f"Error fetching place-recording relationships for {place_id}: {e}")
        return []

# --- Example usage: ---

# 1. Read your list of places from a CSV or other source
places_df = pd.read_csv('places.csv')
place_ids = places_df['id'].tolist()

# 2. Define headers as a dictionary
headers = {
    'User-Agent': 'jihbr@umich.edu'
}

# 3. Define the relationship types to include
relationship_types = [
    "arranged at",
    "engineered at",
    "recorded at",
    "mixed at",
    "edited at",
    "remixed at",
    "produced at",
    "video shot at"
]

# 4. Prepare for checkpointing in Parquet
checkpoint_file = "checkpoint.parquet"
output_data = []

# Try loading an existing checkpoint to skip processed IDs
try:
    checkpoint_df = pd.read_parquet(checkpoint_file)
    processed_place_ids = set(checkpoint_df['place_id'].tolist())
    output_data = checkpoint_df.to_dict('records')  # Continue from checkpoint
    print(f"Loaded {len(output_data)} records from existing checkpoint.")
except (FileNotFoundError, pd.errors.EmptyDataError):
    processed_place_ids = set()

BATCH_SIZE = 1
count_since_checkpoint = 0

for place_id in tqdm(place_ids, desc="Fetching relationships"):
    if place_id in processed_place_ids:
        continue  # Skip already processed place IDs

    relationships = get_place_recording_relationships(place_id, headers, relationship_types)

    # Filter out relationships that do not match the defined types
    filtered_relationships = [
        {
            "type": rel["type"],
            "recording": rel.get("recording", {})
        }
        for rel in relationships
    ]

    output_data.append({'place_id': place_id, 'relationships': filtered_relationships})
    processed_place_ids.add(place_id)
    count_since_checkpoint += 1

    # Check if we've hit our batch threshold for checkpointing
    if count_since_checkpoint >= BATCH_SIZE:
        # Save entire accumulated data so far
        temp_df = pd.DataFrame(output_data)

        # Convert relationships column to JSON strings
        temp_df['relationships'] = temp_df['relationships'].apply(json.dumps)

        temp_df.to_parquet(checkpoint_file, index=False, compression='snappy')
        print(f"Checkpoint saved with {len(output_data)} total records.")
        
        # Reset our counter for the next batch
        count_since_checkpoint = 0

# Finally, save the entire output to a final Parquet file
final_output_df = pd.DataFrame(output_data)

# Convert relationships column to JSON strings for the final write
final_output_df['relationships'] = final_output_df['relationships'].apply(json.dumps)

final_output_df.to_parquet('place_recording_relationships.parquet', 
                           index=False, 
                           compression='snappy')

print(f"Final output saved with {len(output_data)} total records.")


Fetching relationships:   0%|          | 1/69663 [00:01<36:13:54,  1.87s/it]

Checkpoint saved with 1 total records.


Fetching relationships:   0%|          | 2/69663 [00:02<27:27:10,  1.42s/it]

Checkpoint saved with 2 total records.


Fetching relationships:   0%|          | 3/69663 [00:03<21:58:26,  1.14s/it]

...

Checkpoint saved with 40 total records.


Fetching relationships:   0%|          | 41/69663 [00:31<12:58:31,  1.49it/s]

Checkpoint saved with 41 total records.


Fetching relationships:   0%|          | 41/69663 [00:32<15:09:09,  1.28it/s]


KeyboardInterrupt: 